# Exercises for Day 2 of the pyMOR Online Course 2020

## Exercise 1: POD-greedy

In this exercise we will reduce a parametric, time-dependent diffusion-advection-reaction equation using the POD-greedy algorithm, which employs a greedy search in the parameter domain and uses POD to extract low-rank spaces in the time domain.

### Exercise 1 a)

Create a discrete model for the diffusion-advection-reaction equation

$$
\begin{align}
\partial_t u(x,y,t,\mu) - \mu_d \cdot \Delta u(x,y,t,\mu) + \partial_x u(x,y,t,\mu) + \mu_r \cdot u(x,y,t,\mu) &= f(x,y) & (x,y) &\in \Omega := (-1, 1) \times (-1, 1),\ t \in (0, 0.5) \\
u(x,y,t,\mu) &= 0 & (x,y) &\in \partial\Omega,\ t \in (0,0.5) \\
u(x,y,0,\mu) &= u_0(x,y) & (x,y) &\in \Omega
\end{align}
$$

with initial data $u_0$ given by

$$
u_0(x,y) =
\begin{cases}
1 & x^2 + y^2 < 0.04 \\
0 & \text{otherwise}
\end{cases}
$$

and the source term $f$ given by

$$
f(x,y) =
\begin{cases}
1 & (x+0.5)^2 + (y+0.5)^2 < 0.04 \\
0 & \text{otherwise}.
\end{cases}
$$

Specify `parameter_ranges` of $[0.01, 1]$ for $\mu_d$ and of $[0, 100]$ for $\mu_r$. Use continuous finite elements with a `diameter` of $1/100$ and $10$ time steps. Visualize the solution for some combinations of parameter values.

**Hints:**
- Use the `reaction` parameter of `StationaryProblem.__init__` to specify a constant reaction coefficient of value 1.
- Specify the allowed ranges of $\mu_d$ and $\mu_r$ by passing a dict of intervals as the `parameter_ranges` parameter to `InstationaryProblem.__init__`.

### Exercise 1 b)

To compute a basis using POD-greedy, we first need a reductor that assembles an online-efficient error estimator for the ROM. Since the problem is of parabolic type, we can use `ParabolicRBReductor` for that, which will provide an estimator that bounds the error measure

$$
\left[ C_a^{-1}(\mu)\|e_N(\mu)\|^2 + \sum_{n=1}^{N} \Delta t\|e_n(\mu)\|^2_e \right]^{1/2}
$$

where $\|\cdot\|$ denotes the $L^2$-norm, $\|\cdot\|_e$ an energy norm w.r.t. which the bilinear form of the spatial differential operator is coercive, $C_a(\mu)$ is a lower bound for the coercivity constant, $\Delta t$ is the time-step size, $N$ the number of time steps, and $e_n(\mu)$ is the error at time step $n$ for parameter values $\mu$. So, in particular, this quantity is an upper bound for discrete version of the space-time energy error

$$
\left[ \int_0^T \|e(t, \mu)\|^2 \right]^{1/2}.
$$

As the energy norm use the norm induced by the `fom.h1_0_semi_product`, for which the coercivity constant is simply given by $\mu_d$.

In the POD-greedy algorithm we do the following in each iteration:
1. Determine $\mu^*$ for which the estimated space-time error is maximal.
2. Compute the FOM solution $u(\mu^*)$ and the ROM solution $u_{red}(\mu^*)$.
3. Compute the orthogonal-projection $u_{proj}(\mu^*)$ of $u_{red}(\mu^*)$ onto the reduced basis at each time instance.
4. Compute a POD of the projection defects $u_\perp(t, \mu^*) := u_{red}(t, \mu^*) - u_{proj}(t, \mu^*)$.
5. Extend the basis with a certain number of POD modes of $u_\perp$.

This will all happen automatically for you in `rb_greedy` when the algorithm detects that `solve` returns `VectorArrays` with more than one solution vector. By default, one POD mode per iteration will be added to the basis.

Compute a POD-greedy basis for a training set of $20\times 20$ uniformly sampled parameter values. Specify an absolute error tolerance of $10^{-2}$.

**Hints:**
- You need to provide a (lower bound) `coercivity_estimator` for $C_a$ to `ParabolicRBReductor.__init__`. In this case you can use `ProjectionParameterFunctional('diffusion', 1)`.
- Use, `p.parameter_space` to create the training set.

### Exercise 1 c)

Compute the model order reduction error and the estimated error for a test set of 30 randomly sampled parameter values. Determine the maximum and minimum ratio between error and estimated error. Visualize the FOM and ROM solutions as well as their difference for the parameter values maximizing the error. Also compute the ROM speedup.

### Exercise 1 d)

Since `rb_greedy` only adds one POD mode per iteration, it can happen that the same parameter values are selected multiple times during basis generation. Hence, we can save some offline time by caching the FOM solutions.

Enable caching by calling `fom.enable_caching('disk')` and compute the POD-greedy basis again. Compare the timings.

### Exercise 1 e)

Disk-based caching is only possible when the solution `VectorArrays` can be serialized using the `pickle` protocol. When using an external solver, this might not be the case. As an alternative, we can use memory-based caching using `fom.enable_caching('memory')`. Another approach to accelerate the offline phase is to add more than one POD mode to the basis per iteration, possibly at the expense of a slightly bigger final reduced basis. To do this, we need to pass appropriate `pod_modes` in the `extension_params` dict to `rb_greedy`. 

Disable caching again by calling `fom.disable_caching`. Build a new POD-greedy basis by adding 3 POD modes per iteration. Compare the offline times as well as the quality of the ROM on the test_set from part c).


## Exercise 2: Reducing a model with output

So far we have only looked at state-space approximations of the FOM. In this exercise we will build a ROM with an output functional that can be efficiently evaluated without depending on any full-order calculations.

### Exercise 2 a)

In this exercise we will work again with the heat-sink model from Exercise 3 c) from the first exercise sheet. Add the output functional $\ell(u)$ to the model which is given by the average temperature at the base of the heat sink:

$$
\ell(u) := \frac{1}{|\Gamma_b|} \int_{\Gamma_b} u(s)\,\textrm{ds}, \qquad \Gamma_b := \partial\Omega \cap \mathbb{R} \times \{0\}.
$$

Let the constant diffusion coefficient $d$ be a parameter of the model. Plot of the base temperature in dependence of the diffusion coefficient for $d \in [1, 10^5]$.

### Exercise 2 b)

Use `scipy.optimize.bisect` to determine the diffusion coefficient $d \in [1, 10^5]$ for which the base temperature is 45. How many solutions of the FOM are required?

### Exercise 2 c)

Create a reduced basis from 5 logarithmically spaced solution snapshots of the FOM. Manually build a ROM by using the `project` method. Again determine $d$ for base temperature 45, this time using the ROM in the `bisect` call.

### Exercise 2 d)

pyMOR's reductors automatically project the output functional for you. Rebuild the ROM for the same basis using `StationaryRBReductor`. Plot the diffusion coefficient in dependence of the base temperature using repeated `bisect` calls.

**Note:** Of course, this exercise has to be taken with a grain of salt. `bisect` is not a very efficient root-finding algorithm and the default tolerance for convergence is much smaller than the model order reduction error. The main purpose of this exercise was to show that the ROM output can be used as a more efficient drop-in replacement for the FOM output. For computing gradients of the ROM output, e.g. to use a Newton algorithm, see Tim Keil's presentation on Friday.

## Exercise 3: Working with affine spaces

By now we have only considered PDEs with homogeneous Dirichlet boundary conditions, causing the ansatz space for the weak formulation to be a linear space. For problems with non-zero Dirichlet boundaries, the ansatz space will be an affine space. In this exercise we will treat the reduction of problems with affine solution spaces.

### Exercise 3 a)

Create a discrete model for the $2 \times 2$ `thermal_block_problem`. However, change the boundary condition to be given by:

$$
u(x, y) =
\begin{cases}
1 & y = 0 \\
0 & \text{otherwise}
\end{cases}
\qquad\text{for}\qquad
x \in \partial\Omega.
$$

**Hints:**
- First call `thermal_block_problem` to obtain a standard thermal-block problem with homogeneous Dirichlet boundary conditions. Use `with_` to exchange `dirichlet_data` with an appropriate `ExpressionFunction`.

### Exercise 3 b)

Use `to_numpy()` and `np.where` on a solution of the model to determine the degrees of freedom associated with the non-zero Dirichlet boundary. Solve for different parameter values to check that, indeed, all solutions are exactly one at these DOFs.

### Exercise 3 c)

Let's just ignore that the solutions lie in an affine space. Build a ROM from 5 random solution snapshots. Solve FOM and ROM for new parameter values and compute the error. Is the boundary condition fulfilled?

### Exercise 3 d)

We see that, although the boundary condition is not fulfilled exactly, it is fulfilled up to a quite small approximation error. In many cases, it is actually quite feasible to just ignore the fact that the ansatz space is an affine subspace of the discrete function space and work with this entire space. In fact, for many discretization methods, like finite volume methods, Dirichlet boundary conditions are only weakly enforced. However, there are many reasons to work with affine spaces and to enforce the ROM solution to exactly lie in this affine space. In particular:

1. There are many different ways to realize the handling of Dirichlet boundary conditions in PDE solvers, and it is often unclear/unknown how these affect the ROM. This not only affects the ansatz space but also the test space which would normally consist only of functions which are zero at the Dirichlet boundaries. In particular, most implementations retain the degrees of freedom related to the Dirichlet boundaries and modify the system matrix to include an equation forcing the associated degrees of freedom to have the right value. (This is also what is done in pyMOR's builtin discretization toolkit.) In the ROM, these equations will in effect put a penalty on the violation of the boundary conditions. The weighting of this penalty is usually unknown, however. So there is no way of controlling how well the boundary conditions will be fulfilled.

2. Often system matrices are also used to define energy norms. Due to boundary treatment, these matrices often are no longer symmetric even though their corresponding bilinear form is. Using those matrices to orthogonalize functions with non-zero Dirichlet boundaries will cause errors.

3. Sometimes, in particular for non-linear problems, it might be a requirement to choose ansatz functions from an appropriate affine space, since an application of the `Operator` to functions outside this space might cause internal errors inside the PDE solver.

4. For some problems, the ROM equations will become ill-posed when an inappropriate ansatz space is chosen. For instance, solving the Neumann problem for functions with a given non-zero average will render the ROM system matrix singular.

5. Sometimes it may be a user requirement that the Dirichlet constraints are exactly fulfilled.

A standard way to mitigate issues with affine ansatz spaces is to reformulate the problem to have solutions in a linear space. In particular, consider a standard linear `StationaryModel` of the form

$$
 A(\mu)\cdot u(\mu) = f.
$$

Assume that $u_{aff}$ is any function from the affine space, e.g. a function with the correct boundary values, then we can decompose $u(\mu)$ as

$$
 u(\mu) = u_{aff} + u_0(\mu)
$$

and solve

$$
 A(\mu) \cdot [u_{aff} + u_0(\mu)] = f
$$

for $u_0(\mu)$. If we now collect snapshots $u_0(\mu_i)$ and construct a linear subspace $V_N$ from the linear span of these snapshots, then we can build the ROM using ansatz functions from

$$
u_{aff} + V_N
$$

which will be an affine subspace of the affine solution space. In the case of Dirichlet boundary conditions, $V_N$ will only consist of functions with zero boundary values.

We still need to build an online-efficient ROM from our ansatz. In the linear case this is quite straightforward, as we can rewrite the equation system as

$$
A(\mu) \cdot u_0(\mu) = f - A(\mu) \cdot u_{aff}.
$$

This means that we solve an equation system with the same system matrix, but with a modified right-hand side.

Build a `StationaryModel` for this equation system and check that its solutions have zero boundary values. Reduce the modified ROM using snapshot solutions from the original FOM. Reconstruct a full-order function for the ROM solution and compare the error with your earlier ROM solution. Verify that the Dirichlet condition is exactly fulfilled.

**Hints:**
- Use a solution of the FOM for the definition of $u_{aff}$.
- Wrap $u_{aff}$ as an Operator using `VectorOperator`.
- Use `@` to concatenate `fom.operator` with your `VectorOperator` and subtract it from `fom.rhs`.
- Use `fom.with_` to get a new FOM with replaced right-hand side.
- pyMOR should automatically handle the correct offline/online decomposition of the new right-hand side when projecting (check that).